In [2]:
import pandas as pd 
import scanpy as sc
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

scr_path = '/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/snippets'
sys.path.append(scr_path)
import singleCellFun as sf

import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Read adult data

In [4]:
# Define the data as a dictionary
data = {
    "Hong et al. 2023": [13061, 4490, 1580],
    "Pu et al. 2021": [1868, 2250, 1906],
    "Wang et al. 2022": [441, 494, 711],
    "Lu et al. 2023": [389, 233, 132],
    "Mosteiro et al. 2023": [5642, 6705, 5445],
#     "Han et al. 2020": [1105, 1053, 1161]
}

# Define row labels
index = ["fTFC1", "ambiguous", "fTFC2"]

# Create DataFrame
adult_df = pd.DataFrame(data, index=index)

# Display the DataFrame
adult_df


,Hong et al. 2023,Pu et al. 2021,Wang et al. 2022,Lu et al. 2023,Mosteiro et al. 2023
fTFC1,13061,1868,441,389,5642
ambiguous,4490,2250,494,233,6705
fTFC2,1580,1906,711,132,5445


In [5]:
adult_df.sum()

Hong et al. 2023        19131
Pu et al. 2021           6024
Wang et al. 2022         1646
Lu et al. 2023            754
Mosteiro et al. 2023    17792
dtype: int64

In [33]:
adult_df.loc['fTFC2'] / adult_df.sum() * 100
# # get percent 
# dic = '{:.2f}'
# df_percent = df2plot / df2plot.sum() * 100
# print(f"fTFC1 %: {dic.format(df_percent.loc['fTFC1'].min())}% - {dic.format(df_percent.loc['fTFC1'].max())}%")
# print(f"fTFC2 %: {dic.format(df_percent.loc['fTFC2'].min())}% - {dic.format(df_percent.loc['fTFC2'].max())}%")

Hong et al. 2023         8.258847
Pu et al. 2021          31.640106
Wang et al. 2022        43.195626
Lu et al. 2023          17.506631
Mosteiro et al. 2023    30.603642
dtype: float64

# Read fetal data

In [6]:
# load single cell data
adata = sc.read('/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/THY2023_Jun_2n/soupX_2n/Thyrocytes/workingObj/cdata_thy_noStressEmb.h5ad')
adata.shape

(50597, 28923)

In [7]:
fetal_df = sf.summaryTable(adata,['celltype','donor'], return_df=True)
fetal_df[fetal_df['celltype'] == 'thy_Lumen-forming']

[INFO] - total cells: 50597


,celltype,donor,cells
12,thy_Lumen-forming,Hrv194,1057
17,thy_Lumen-forming,Hrv202,917
20,thy_Lumen-forming,Hrv197,827
22,thy_Lumen-forming,Hrv218,796
23,thy_Lumen-forming,Hrv214,791
24,thy_Lumen-forming,Hrv196,748
27,thy_Lumen-forming,Hrv199,677
28,thy_Lumen-forming,Hrv203,640
29,thy_Lumen-forming,Hrv188,601
30,thy_Lumen-forming,Srv10,598


In [8]:
adata.obs['celltype'].value_counts()

celltype
thy_TH_processing    28998
thy_Lumen-forming    11720
thy_Cycling           9879
Name: count, dtype: int64

# Agg fTFC2 numbers

In [26]:
fTFC2_num = fetal_df.loc[fetal_df['celltype'] == 'thy_Lumen-forming', 'cells'].values.tolist() # number of fTFC2 per donor
f_sum = fetal_df.groupby('donor')['cells'].sum().values.tolist() #total number of cell per fetal donor 

aTFC2_num = adult_df.loc['fTFC2'].values.tolist() # number of aTFC2 per donor
a_sum = adult_df.sum().values.tolist() #total number of cell adult donor 

In [27]:
# Count successes and total observations
# count = [group_a.sum(), group_b.sum()]  # [5, 8]
# nobs = [len(group_a), len(group_b)]     # [10, 10]

count = [np.sum(fTFC2_num), np.sum(aTFC2_num)]
nobs = [np.sum(f_sum), np.sum(a_sum)]

# Run z-test
stat, pval = proportions_ztest(count, nobs)
print(f"Z-statistic: {stat:.3f}, p-value: {pval:.6f}")

Z-statistic: 5.970, p-value: 0.000000


In [28]:
pval < 1e-4

True

In [25]:
aTFC2_num

[1580, 1906, 711, 132, 5445]